In [478]:
import threading
import queue
import time
import random
import numpy as np
import concurrent.futures

In [555]:

N=5
W=4
U=3
V=2

# properties of any function
Fun_name=['cpu','mem gig','Disk','BW','deadline msec']
Function_Resource=np.array([[0,0,0,0,0,0],
              [0,1/5, 128/5120, 128/10240, 128/4096, 10],
          [0,2/5, 256/5120, 128/10240, 256/4096 ,20],
          [0,4/5 ,128/5120 ,256/10240 ,256/4096, 40],
          [0,1/5, 100/5120 ,10/10240  ,10/4096 ,80],
          [0,2/5 ,200/5120, 20/10240, 10/4096, 160]])

#Resourse of edge(1-3) and cloud(4-5)
Edge_Cloud_name=['cpu','mem','Disk','BW']
Edge_Cloud_Resource= np.array([[0,0,0,0,0],
                      [0,.2, 1024/5120, 1024/10240 ,1024/4096],
                      [0,.4, 512/5120, 1024/10240, 2048/4096],
                      [0,.2 ,256/5120, 1024/10240, 4096/4096],
                      [0,.8 ,2049/5120, 10240/10240, 4096/4096],
                      [0,1 ,5120/5120,10240/10240, 4096/4096]])
Edge_Cloud_Remind=Edge_Cloud_Resource

#Cost of device edge(1-3) cloude(4-5)
Edge_Cloud_Cost=np.array([[0,0,0,0,0],
                          [0,10,10,5,5],
                          [0,10,10,5,5],
                          [0,10,10,5,5],
                          [0,30,30,10,10],
                          [0,30,30,10,10]])

#lambda terminal j of function i(terminal=row(1-4)), (function=column(1-5))                  
Terminal_Fun_Lambda=np.array([[0,0,0,0,0,0],
                        [0,1,2,10,3,4],
                        [0,1,2,3,4,5],
                        [0,0.4,3,2,1,10],
                        [0,4,10,3,2,1]])
E_D_Lambda=np.array([0,10,5,10])
C_D_Lambda=np.array([0,20,20])

#Connection delay Devices to Terminal(row: edge(1-3) cloud(4-5), terminal: column(1-4))
Connection_Latency=np.array([[0,0,0,0,0,0],
                       [0,1, 2, 1, 10, 20], [0,1.5, 2.5, 1.5, 10, 20], [0,3, 1, 2, 15, 15], [0,2, 2, 1, 12, 16]])

utilization=np.array([0,0,0,0,0])                       
# boolian variables
y=np.zeros((U,W,N),dtype=int)
z=np.zeros((V,W,N),dtype=int)

# constant for cost
A = 1
B = 5
D = 10
H = 100
# Cost of any function on any Device(row: edge(1-3) cloud(4-5), function: column(1-5))
Cost_Device_Function=np.zeros((U+V+1,N+1),dtype=float)
# MU of any function on any Device(row: edge(1-3) cloud(4-5), function: column(1-5))
MU_Device_Function=np.zeros((U+V+1,N+1),dtype=float)
func_res_runtim = [0,.002, .004, .008, .001, .004]

# number of executed functions
M=10

#Total Delay
Delay=0

# Utilization of any device
UTI_Device=np.zeros((U+V+1),dtype=float)
Cost_function=np.zeros(U+V+1,dtype=float)
#Total Cost
Cost=0

#Tproc terminal j of function i(terminal=row(1-4)), (function=column(1-5))                  
Tproc=np.zeros((U+V+1),dtype=float)
Latency=np.zeros((U+V+1),dtype=float)
Cost_Function=np.zeros((U+V+1),dtype=float)
Execution_Time=np.zeros((U+V+1),dtype=float)
                        
costlist_edg = [(10 / A), (10 / B), (5 / D), (5 / H)]
costlist_server = [(30 / A), (30 / B), (10 / D), (10 / H)]                        
                        
                    

In [556]:
#محاسبه ی هزینه ی اجرای توابع روی دستگاههای مختلف
def Cost_DF():
    
    for j in range(1,U+V+1):
        for i in range (1,N+1):
            if (j<=3):
                Cost_Device_Function[j,i]=(costlist_edg[0]*Function_Resource[i,1]) + (costlist_edg[1]*Function_Resource[i,2])+(costlist_edg[2]*Function_Resource[i,3])+(costlist_edg[3]*Function_Resource[i,4])
            else:
                Cost_Device_Function[j,i]=(costlist_server[0]*Function_Resource[i,1]) + (costlist_server[1]*Function_Resource[i,2])+(costlist_server[2]*Function_Resource[i,3])+(costlist_server[3]*Function_Resource[i,4])
    print(Cost_Device_Function) 
   
Cost_DF()

[[ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          2.0578125   4.109375    8.065625    2.03967285  4.07922363]
 [ 0.          2.0578125   4.109375    8.065625    2.03967285  4.07922363]
 [ 0.          2.0578125   4.109375    8.065625    2.03967285  4.07922363]
 [ 0.          6.165625   12.31875    24.18125     6.1184082  12.23657227]
 [ 0.          6.165625   12.31875    24.18125     6.1184082  12.23657227]]


In [557]:
#محاسبه ی زمان اجرای هر تابع در دستگاه های مختلف بر اساس پردازشگر
def Cal_MU():
    for j in range(1,U+V+1):
        for i in range (1,N+1):
            MU_Device_Function[j,i]=func_res_runtim[i]
    print(MU_Device_Function)        
    

Cal_MU()


[[0.    0.    0.    0.    0.    0.   ]
 [0.    0.002 0.004 0.008 0.001 0.004]
 [0.    0.002 0.004 0.008 0.001 0.004]
 [0.    0.002 0.004 0.008 0.001 0.004]
 [0.    0.002 0.004 0.008 0.001 0.004]
 [0.    0.002 0.004 0.008 0.001 0.004]]


In [558]:
# queques
Q=queue.Queue()
q_E1=queue.Queue()
q_E2=queue.Queue()
q_E3=queue.Queue()
q_C1=queue.Queue()
q_C2=queue.Queue()

In [559]:
# تولید درخواست تابع از دستگاههای ترمینال
def produce_function(id_fun, id_Dv, delay):
    while Q.qsize()<M:
        # اضافه کردن عنصر به صف
        Q.put((id_fun, id_Dv, time.time()))
        print(f"Function {id_fun} of {id_Dv} produced at {time.time()}")
        # تاخیر تصادفی
        time.sleep(delay)

In [560]:
# تعیین تعداد نخ‌ها
num_threads = 20

# تعیین زمان اجرای هر نخ
thread_delay = 0.5  # زمان تاخیر به ثانیه



# تعریف تابع برای ایجاد نخ‌ها
def start_producers():
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        # ایجاد تمام نخ‌ها به صورت همزمان
        futures = {executor.submit(produce_function, i % 5 + 1, i % 4 + 1, Terminal_Fun_Lambda[i % 4 + 1, i % 5 + 1]): i for i in range(1, num_threads + 1)}
        # منتظر ماندن تا اتمام همه نخ‌ها
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as exc:
                print(f'generated an exception: {exc}')



In [561]:
# اجرای توابع
start_time = time.time()
start_producers()
# اجرای برنامه به مدت 1 ثانیه
time.sleep(1)

Function 2 of 2 produced at 1722865192.0980046
Function 3 of 3 produced at 1722865192.0997446
Function 4 of 4 produced at 1722865192.0997446
Function 5 of 1 produced at 1722865192.0997446
Function 1 of 2 produced at 1722865192.100745
Function 2 of 3 produced at 1722865192.100745
Function 3 of 4 produced at 1722865192.1012495
Function 4 of 1 produced at 1722865192.1012495
Function 5 of 2 produced at 1722865192.1018448
Function 1 of 3 produced at 1722865192.1018448


In [562]:
import threading

def Exe_in_ED1():
    while q_E1.qsize()>0:
        (i,j,a_t)=q_E1.get()
        Tproc[1]=1/(0.0001+(MU_Device_Function[j,i]-(1/E_D_Lambda[1])))
        Latency[1]=2*Connection_Latency[1,j]
        Execution_Time[1]=2*Latency[1]+MU_Device_Function[j,i] #Tproc[1]
        Cost_Function[1]+=Cost_Device_Function[1,i]
        utilization[0]+=1
        Edge_Cloud_Remind[1,1]+=Function_Resource[i,1]
        Edge_Cloud_Remind[1,2]+=Function_Resource[i,2]
        Edge_Cloud_Remind[1,3]+=Function_Resource[i,3]
        Edge_Cloud_Remind[1,4]+=Function_Resource[i,4]
        
    pass

def Exe_in_ED2():
    while q_E2.qsize()>0:
        (i,j,a_t)=q_E2.get()
        Tproc[2]=1/(0.0001+(MU_Device_Function[j,i]-(1/E_D_Lambda[2])))
        Latency[2]=2*Connection_Latency[2,j]
        Execution_Time[2]=2*Latency[2]+MU_Device_Function[j,i] #Tproc[2]
        Cost_Function[2]+=Cost_Device_Function[2,i]
        utilization[1]+=1
        Edge_Cloud_Remind[2,1]+=Function_Resource[i,1]
        Edge_Cloud_Remind[2,2]+=Function_Resource[i,2]
        Edge_Cloud_Remind[2,3]+=Function_Resource[i,3]
        Edge_Cloud_Remind[2,4]+=Function_Resource[i,4]
        
    pass

def Exe_in_ED3():
    while q_E3.qsize()>0:
        (i,j,a_t)=q_E3.get()
        Tproc[3]=1/(0.0001+(MU_Device_Function[j,i]-(1/E_D_Lambda[3])))
        Latency[3]=2*Connection_Latency[3,j]
        Execution_Time[3]=2*Latency[3]+MU_Device_Function[j,i] #Tproc[3]
        Cost_Function[3]+=Cost_Device_Function[3,i]
        utilization[2]+=1
        Edge_Cloud_Remind[3,1]+=Function_Resource[i,1]
        Edge_Cloud_Remind[3,2]+=Function_Resource[i,2]
        Edge_Cloud_Remind[3,3]+=Function_Resource[i,3]
        Edge_Cloud_Remind[3,4]+=Function_Resource[i,4]
    pass

def Exe_in_CD1():
    while q_C1.qsize()>0:
        (i,j,a_t)=q_C1.get()
        Tproc[4]=1/(0.0001+(MU_Device_Function[j,i]-(1/C_D_Lambda[1])))
        Latency[4]=2*Connection_Latency[4,j]
        Execution_Time[4]=2*Latency[4]+MU_Device_Function[j,i] #Tproc[4]
        Cost_Function[4]+=Cost_Device_Function[4,i]
        Edge_Cloud_Remind[4,1]+=Function_Resource[i,1]
        Edge_Cloud_Remind[4,2]+=Function_Resource[i,2]
        Edge_Cloud_Remind[4,3]+=Function_Resource[i,3]
        Edge_Cloud_Remind[4,4]+=Function_Resource[i,4]
    pass

def Exe_in_CD2():
    while q_C2.qsize()>0:
        (i,j,a_t)=q_C2.get()
        Tproc[5]=1/(0.0001+(MU_Device_Function[j,i]-(1/C_D_Lambda[2])))
        Latency[5]=2*Connection_Latency[5,j]
        Execution_Time[5]=2*Latency[5]+MU_Device_Function[j,i] #Tproc[5]
        Cost_Function[5]+=Cost_Device_Function[5,i]
        Edge_Cloud_Remind[5,1]+=Function_Resource[i,1]
        Edge_Cloud_Remind[5,2]+=Function_Resource[i,2]
        Edge_Cloud_Remind[5,3]+=Function_Resource[i,3]
        Edge_Cloud_Remind[5,4]+=Function_Resource[i,4]
    pass

       
def random_agent():
    while Q.qsize()>=1:
        (f,d,t)=Q.get()
        print(f)
        j=random.randint(1,5)
        if (j==1):
            q_E1.put((f,d,t))
            
            
            #E_D_Lambda[1]+=Terminal_Fun_Lambda[d,f]
            Edge_Cloud_Remind[1,1]-=Function_Resource[f,1]
            Edge_Cloud_Remind[1,2]-=Function_Resource[f,2]
            Edge_Cloud_Remind[1,3]-=Function_Resource[f,3]
            Edge_Cloud_Remind[1,4]-=Function_Resource[f,4]
        elif (j==2):
            q_E2.put((f,d,t))
            
            
            #E_D_Lambda[2]+=Terminal_Fun_Lambda[d,f]
            Edge_Cloud_Remind[2,1]-=Function_Resource[f,1]
            Edge_Cloud_Remind[2,2]-=Function_Resource[f,2]
            Edge_Cloud_Remind[2,3]-=Function_Resource[f,3]
            Edge_Cloud_Remind[2,4]-=Function_Resource[f,4]
        elif(j==3):
            q_E3.put((f,d,t))
            
            
            #E_D_Lambda[3]+=Terminal_Fun_Lambda[d,f]
            Edge_Cloud_Remind[3,1]-=Function_Resource[f,1]
            Edge_Cloud_Remind[3,2]-=Function_Resource[f,2]
            Edge_Cloud_Remind[3,3]-=Function_Resource[f,3]
            Edge_Cloud_Remind[3,4]-=Function_Resource[f,4]
        elif(j==4):
            q_C1.put((f,d,t))
            
            #E_D_Lambda[1]+=Terminal_Fun_Lambda[d,f]
            Edge_Cloud_Remind[4,1]-=Function_Resource[f,1]
            Edge_Cloud_Remind[4,2]-=Function_Resource[f,2]
            Edge_Cloud_Remind[4,3]-=Function_Resource[f,3]
            Edge_Cloud_Remind[4,4]-=Function_Resource[f,4]
        elif(j==5):
            q_C2.put((f,d,t))
            
            #C_D_Lambda[2]+=Terminal_Fun_Lambda[d,f]
            Edge_Cloud_Remind[5,1]-=Function_Resource[f,1]
            Edge_Cloud_Remind[5,2]-=Function_Resource[f,2]
            Edge_Cloud_Remind[5,3]-=Function_Resource[f,3]
            Edge_Cloud_Remind[5,4]-=Function_Resource[f,4]     
        pass

def main():
    # Create threads for each function
    thread_random=threading.Thread(target=random_agent)
    thread_ED1 = threading.Thread(target=Exe_in_ED1)
    thread_ED2 = threading.Thread(target=Exe_in_ED2)
    thread_ED3 = threading.Thread(target=Exe_in_ED3)
    thread_CD1 = threading.Thread(target=Exe_in_CD1)
    thread_CD2 = threading.Thread(target=Exe_in_CD2)
  

    # Start the threads
    thread_random.start()
    thread_ED1.start()
    thread_ED2.start()
    thread_ED3.start()
    thread_CD1.start()
    thread_CD2.start()

    # Wait for all threads to finish
    thread_random.join()
    thread_ED1.join()
    thread_ED2.join()
    thread_ED3.join()
    thread_CD1.join()
    thread_CD2.join()

__name__="__main__"
if __name__ == "__main__": 
    main()

Exception in thread Thread-220 (Exe_in_CD2):
Traceback (most recent call last):
  File "C:\Program Files\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\A.Zahed\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\A.Zahed\AppData\Local\Temp\ipykernel_15296\2228327869.py", line 64, in Exe_in_CD2
IndexError: index 5 is out of bounds for axis 0 with size 5


2
3
4
5
1
2
3
4
5
1


In [563]:
print(q_E1.qsize())
print(q_E2.qsize())
print(q_E3.qsize())
print(q_C1.qsize())
print(q_C2.qsize())

0
0
0
0
1


In [564]:
print(Tproc[1])
print(Latency[1])
print(Execution_Time[1])
print(Cost_Function[1])

-10.427528675703858
4.0
8.004
4.109374999999999


In [565]:

print(Tproc[2])
print(Latency[2])
print(Execution_Time[2])
print(Cost_Function[2])


-5.027652086475616
3.0
6.001
6.149047851562499


In [566]:

print(Tproc[3])
print(Latency[3])
print(Execution_Time[3])
print(Cost_Function[3])

-10.214504596527068
4.0
8.002
6.1370361328125


In [567]:

print(Tproc[4])
print(Latency[4])
print(Execution_Time[4])
print(Cost_Function[4])

-20.876826722338205
4.0
8.002
24.52060546875


In [568]:

print(Tproc[5])
print(Latency[5])
print(Execution_Time[5])
print(Cost_Function[5])

-23.86634844868735
0.0
0.0
0.0


In [569]:
uti=0
for t in range(1,5):
    Cost+=Cost_Function[t]
    Delay+=Execution_Time[t]
    uti+=utilization[t-1]
    
Delay=Delay/M
Cost=Cost/M
uti=uti/M
print(Cost)
print(Delay)
print(uti)

4.091606445312499
3.0009
0.5
